In [ ]:
# Get the data from Kaggle:
# !kaggle datasets download -d vishalsubbiah/pokemon-images-and-types

# !mkdir -p ../input/pokemon-images-and-types
# !unzip pokemon-images-and-types.zip -d ../input/pokemon-images-and-types

# Explore the data

Read the images and metadata information, and display examples of images.

In [ ]:
import os
import pandas as pd

INPUT_DIR = '../input/pokemon-images-and-types'

pokemon = pd.read_csv(os.path.join(INPUT_DIR, 'pokemon.csv'))
pokemon = pokemon.sort_values(by='Name')

display(pokemon.head())
display(pokemon.describe())
display(pokemon['Type1'].unique())

In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def read_img(img_name):
    basename = os.path.basename(img_name)
    
    name, suffix = basename.split('.')
    mdata = matplotlib.image.imread(img_name)
    
    if suffix != 'png':
        mdata = mdata.astype(np.float32) / 255
   
    return (mdata[:,:,:3], name)

mdata, label = read_img(os.path.join(INPUT_DIR, 'images/images/cutiefly.jpg'))

print(f'printing image of {label}')
type(mdata)
display(mdata.shape)
plt.imshow(mdata)

# Preprocessing Data

Create data preprocessing functions that reads in the following data:

- Label data with "Type1" of pokemon metadata, with OneHotEncoding.
- The image data from JPG and PNG files in the images/images directory.

The output data uses a generator to return `(image_data, label)` tuples, and creates `tf.data.Dataset`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
type1 = encoder.fit_transform(pokemon['Type1'].values.reshape((-1, 1)))

display(encoder.categories_)
display(len(encoder.categories_[0]))

In [ ]:
display(type1)

print(encoder.inverse_transform(type1)[0])

In [ ]:
def get_type1name(name):
    return pokemon[pokemon['Name'] == name]['Type1'].values[0]

def encode_type(type_name):
    return encoder.transform(type_name).toarray()

print('Type1 of pokemon abomasnow:', get_type1name('abomasnow'))
print('Type1 encoding of Grass:', encode_type([['Grass']]))

In [ ]:
# Get the result label information

type1 = encode_type(pokemon['Type1'].values.reshape((-1,1)))

In [ ]:
# Create generator for result image data

import os

def gen_train_imgs():
    imglist = os.listdir(os.path.join(INPUT_DIR, 'images/images'))
    
    for i, img in enumerate(imglist):
        imgdata, name = read_img(os.path.join(INPUT_DIR, 'images/images', img))
        type_label = type1[i]
        
        yield (imgdata, type_label)

In [ ]:
# Test the generator

g = gen_train_imgs()

for i in range(10):
    mdata, label = next(g)
    plt.imshow(mdata)
    plt.show()

In [ ]:
import tensorflow as tf

dataset = tf.data.Dataset.from_generator(
    gen_train_imgs,
    output_signature=(
        tf.TensorSpec(shape=(120,120,3), dtype=tf.float32),
        tf.TensorSpec(shape=(18,), dtype=tf.int32),
    ),
)

In [ ]:
# split the train validation
#batch_size = 4
#nbatch = int(pokemon.shape[0] / 4)
data_size = pokemon.shape[0]

train_size = int(data_size * 0.8)
val_size = data_size - train_size

print(f'Splitting data size of {data_size} to train: {train_size} and val: {val_size}')
dataset = dataset.shuffle(32)

train_data = dataset.take(train_size).batch(4)
val_data = dataset.skip(train_size).take(val_size).batch(4)

# Create Tensorflow Model

Create CNN model with Tensorflow Keras library, with Convolutional, Maxpooling layers, and output the result vector the shape of (# of classes).

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

def build_model():
    model = tf.keras.Sequential([
        Conv2D(16, 3, 3, input_shape=(120, 120, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(18, activation='softmax'),
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])
    
    return model

model = build_model()
model.summary()

In [ ]:
history = model.fit(
    train_data, 
    epochs=80,
    validation_data=val_data,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()